# Ichi Told Me

### Assumptions:

It is difficult to catch every top and bottom of a market.
>That is where the smart money is shaking out the weak hands.           https://www.youtube.com/watch?v=TzKQwttv9IA

Ichimoku is a full fleged trading system.
>The cloud system is designed to capturing 80% of price movements (I.e. 80/20 rule)
https://www.linkedin.com/pulse/ichimoku-cloud-jonathan-camphin/

Rebalancing preforms ~63% better than HODL on trending and ranging markets.

AND
>Rebalancing once every hour resulted in a median portfolio performance boost of 94% over buy-and-hold. 83.0% of all portfolios which used hourly rebalances performed better than buy-and-hold.
https://blog.shrimpy.io/blog/cryptocurrency-portfolio-rebalancing-bittrex-analysis

AND

>A 15% threshold rebalance outperformed HODL by 305%.
https://blog.shrimpy.io/blog/the-best-threshold-for-cryptocurrency-rebalancing-strategies

### Goal:

__Identify strong to medium Ichimoku signals on a 1 hour timeframe to create dynamic portfolios that can be rebalanced hourly or at a percent deviation threshold on https://www.shrimpy.io/.__

### Tools:

#### Languages: 
Python

#### Visuals:
DASH by Plotly https://dash.plot.ly

#### APIs: 
Shrimpy Developer API  https://developers.shrimpy.io/docs/#introduction

#### Deployment:
[DigitalOcean](https://www.digitalocean.com/) or [Vultr](https://www.vultr.com/) servers.

____

## RULES

These rules are more conservative than most and are designed to nearly eliminate false
signals.

https://www.youtube.com/watch?v=X5lxBTrN-Yk&t=1318s

    CLOSE ABOVE CLOUD
    Close > Kijun_Sen & Close > Tenkan_Sen 
    
    CONVERSION LINE IS GREATER THAN BASE LINE
    Tenkan_Sen > Kijun_Sen
    
    *KEY
    CLOUD AHEAD IS BULLISH (GREEN) *look displacement ahead
    cloud.shift(displacement) == green 
    
    *KEY
    LAGGIN SPAN IS GREATER OR CROSSES ABOVE CLOUD  *look displacement behind
    Chikou_Span > cloud.shift(-displacement)

____

# TODO 

#### (move this cell to above the next step to be completed)

## Get Supported Exchanges
Only required when scaling to other exchanges.

In [26]:
# HTTP Request
# GET https://dev-api.shrimpy.io/v1/list_exchanges

def list_exchanges():
    res = requests.get("https://dev-api.shrimpy.io/v1/list_exchanges")
    return json_normalize(res.json())

## Get Trading Pairs

Pick and exchange to start with.
Binance is popular and has a lot of trading volume and pairs.
However, Binance like someother exchanges, will discontinue service to US customers soon.
https://www.theblockcrypto.com/2019/06/14/us-customers-to-be-blocked-from-trading-on-binance-com/

In [ ]:
# HTTP Request
# GET https://dev-api.shrimpy.io/v1/exchanges/<exchange>/trading_pairs

## Get Candles

In [7]:
# HTTP Request
# GET https://dev-api.shrimpy.io/v1/exchanges/<exchange>/candles

def candles(exchange, quoteTradingSymbol, baseTradingSymbol, interval):
    response = requests.get(f"https://dev-api.shrimpy.io/v1/exchanges/{exchange}/candles?quoteTradingSymbol={quoteTradingSymbol}&baseTradingSymbol={baseTradingSymbol}&interval={interval}")
    return json_normalize(response.json())

## Create Ichimoku data points

Unique to the Ichimoku system is that part of the signal is projected ahead in time.

In [ ]:
# From stackoverflow

# Needs adjusted to accept 'crypto' settings

# TODO
#     build ability to adjust settings based on argument i.e. 
#         if settings == 'default':
#             t1=9, t2=26, t3=52,
#         elif settings == 'crytpo':
#             t1=10, t2=30, t3=60,
#         else:
#             t1=20, t2=60, t3=120

# Turn into function


from datetime import timedelta

high_9 = df['high'].rolling(window= 9).max()
low_9 = df['low'].rolling(window= 9).min()
df['tenkan_sen'] = (high_9 + low_9) /2

high_26 = df['high'].rolling(window= 26).max()
low_26 = df['low'].rolling(window= 26).min()
df['kijun_sen'] = (high_26 + low_26) /2

# this is to extend the 'df' in future for 26 days
# the 'df' here is numerical indexed df
last_index = df.iloc[-1:].index[0]
last_date = pd.to_datetime(df['time']).iloc[-1]
for i in range(26):
    df.loc[last_index+1 +i, 'Date'] = last_date + timedelta(days=i)

df['senkou_span_a'] = ((df['tenkan_sen'] + df['kijun_sen']) / 2).shift(26)

high_52 = df['high'].rolling(window= 52).max()
low_52 = df['low'].rolling(window= 52).min()
df['senkou_span_b'] = ((high_52 + low_52) /2).shift(26)

# most charting softwares dont plot this line
df['chikou_span'] = df['close'].shift(-22) #sometimes -26 

# TODO
#     Edit fill to represent bullish (green) and bearish (red) clouds 
#     between the leading spans.

tmp = df[['close','senkou_span_a','senkou_span_b','kijun_sen','tenkan_sen', 'chikou_span']].tail(300)
a1 = tmp.plot(figsize=(15,10))
a1.fill_between(tmp.index, tmp.senkou_span_a, tmp.senkou_span_b);

## Create portfolio

In [ ]:
# Check ichi signals on the 1 hour timeframe.
# Create a list of coins to be included.

def create_portfolio(exchange, interval):   #TODO
                                            # interval set to 1h unless 
                                            # MAXIMUM_PORTFOLIO_SIZE has been reached,
                                            # then resample to 4h. 
    portfolio = []
    df = {}
    
    coinlist = ticker(exchange).symbol.tolist()
    print('Checking on ' + str(len(coinlist)) + ' coins.')
    
    for coin in coinlist:
        
        try:
            
            df = check_coin(exchange, coin, interval)
            
            if df['condition'][-1:].values == 'buy':
                portfolio.append(coin)
                print('Added ' + coin + ' to portfolio.')
            else: 
                pass

        except:
            print(coin + ' is not availalbe.')
        
#         if df['condition'][-1:].values == 'buy':
#             portfolio.append(coin)
#             print('Added ' + coin + ' to portfolio.')

    return portfolio

## Optimize portfolio

The plan here is to maximize the portolio proformance based on the probability that a 
stong trend will continue.

Do this by checking the ichi signals on a second (Higher) timeframes and sorting by 
those that are on both timeframes and truncate the list at the `MAXIMUM_PORTFOLIO_SIZE`.

The coins that are on both timeframes with be at the top of the list while the others will be lower.

In [ ]:
# Could combine or included in 'create_portfolio' function.

def optimize_portfolio(portfolio):
    print(f"Portfolio has {len(portfolio)} coins in it.")
    if len(portfolio) >= MAXIMUM_PORTFOLIO_SIZE + 1:
        # create_portfolio(exchange, '4h')
        print("That's too many")

## Distribute Portfolio Allocations
Could combine or included in `create_portfolio` function.


In [ ]:
# Distributes coins evenly and takes into account 'BNB' coin to reduce trade fees.

def distribute_allocations(coins):
    
    # create empty list
    allocations = []
    coins_plus = []
    
    # remove duplicates and turn all coins to uppercase
    coins = list(dict.fromkeys([x.upper() for x in coins]))
    
    #TODO
    
    # if len(coins) > coin_limit:
    #   coins = limit_coins(coins)
        
    
    # code if 'BNB' is in the list
    if 'BNB' in coins:
        a = round(100/len(coins),2)
        r = round(100 - a * len(coins),2)
    
        # loop through the coins in the list
        for i in coins:
            
            # Special rules for 'BNB'
            if 'BNB' in coins:
                if i == 'BNB':
                    b = {"symbol" : i , "percent" : round((a + r),2) }
                    allocations.append(b)

                else:    
                    b = {"symbol" : i , "percent" : a }
                    allocations.append(b)

    # code if 'BNB' is NOT in the list
    if 'BNB' not in coins:
#         a = round(100/len(coins),2)
#         r = round(100 - a * len(coins),2)
        
        if r == 0:
            coins_plus = coins
            coinscoins_plus.append('BNB')
        
#         if r == 0:
#             a = round(97/len(coins),2)
#             r = round(97 - a * len(coins),2)
        
        # add 'r' amount of 'BNB' to list before looping
        allocations.append({"symbol" : 'BNB' , "percent" : r + 3})
        
        # loop through the coins in the list
        for i in coins_plus:
            b = {"symbol" : i , "percent" : a }
            allocations.append(b) 

    print(a*len(coins)+r, r)
          
    return allocations
    
    #if remander is not 0:
        

____

# Sending it all to Shrimpy

## Creating and Signing a Request
https://developers.shrimpy.io/docs/#creating-a-request

https://developers.shrimpy.io/docs/#signing-a-request

In [ ]:
# # snipet from shrimpy reddit
# https://www.reddit.com/r/ShrimpyApp/comments/be6o26/shrimpy_api_python_36/
    
import requests, json, time, datetime, base64, hmac, hashlib

key = 'keykeykeykeykeykeykeykeykeykeykeykey'
secret = 'secretsecretsecretsecretsecretsecretsecretsecretsecretsecretsecretsecretsecretsecretsecretsecretsecret'
nonce = str(int(time.time()))

endpoint = '/v1/accounts/'

signurl = (endpoint + 'GET'+nonce).encode('utf-8')

signing = hmac.new(base64.b64decode(secret), signurl , hashlib.sha256)

signing_b64 = base64.b64encode(signing.digest()).decode('utf-8')

header = {'content-type': 'application/json', 
          'SHRIMPY-API-KEY': key,
            'SHRIMPY-API-NONCE' : nonce,
            'SHRIMPY-API-SIGNATURE': signing_b64 }


r = requests.get('https://api.shrimpy.io' + endpoint, headers = header)
print(r)
print(r.text)

____

# See if it works.

## Run Backtest
This endpoint runs a backtest with the supplied settings. 
This endpoint has a rate limit of __20 requests per minute__.

To run a backtest on each iteration of the portolios (changing often), a list of 
optimized porfolios must be created with `startTime` and `endTime` values.
Use the last `usdValue` as the `initialValue` for the next iteration.

In [35]:
# HTTP Request
# POST https://dev-api.shrimpy.io/v1/analytics/backtest/<exchange>/run

____

# Next

##  Set the Strategy

This endpoint sets the strategy for the exchange account. All future rebalance operations will use the new strategy. 

`portfolio = allocations` in request body

In [32]:
# HTTP Request
# POST https://dev-api.shrimpy.io/v1/users/<userId>/accounts/<exchangeAccountId>/strategy

## Set the Rebalance Period
This endpoint sets the rebalance period for the specified exchange account. The next rebalance will be scheduled from the current time. For example, it is currently 4PM and a rebalance period of 24 hours is set, the next rebalance operation will occur at 4PM tomorrow.

In [30]:
# HTTP Request
# POST https://dev-api.shrimpy.io/v1/users/<userId>/accounts/<exchangeAccountId>/rebalance_period

# Set to 1h, but reset to makes sure the proper rebalance period is active.


## Queue Rebalance
This endpoint queues a rebalance for the specified exchange account. When the rebalance occurs, trades will be made so that the linked exchange accounts assets match the current strategy. That is, any assets that are overallocated will be sold and any assets that are underallocated will be purchased.

In [14]:
# HTTP Request
# POST https://dev-api.shrimpy.io/v1/users/<userId>/accounts/<exchangeAccountId>/rebalance

## Get Balance
This endpoint retrieves detailed balance data for an exchange account. By default, the most recent balance is returned. Balance is retrieved from the exchange every 15 minutes for each account, as well as immediately after rebalance operations and trade operations.

In [37]:
# HTTP Request
# GET https://dev-api.shrimpy.io/v1/users/<userId>/accounts/<exchangeAccountId>/balance

____

# Additional features / thoughts

## Sorting portfolio

    Consider another metric to sort the optimized portfolio.
        - OBV On by Volume
        - ATR Average true Range
        - ADX Average Directional Index
        - Stochastics
        - **other**
        
    When to quote in USD (or stable coin) vs. BTC
        if BTC_uptrend:
         quote = USDT
        elif BTC_downtend:
         quote = BTC
        else :
         quote = [“USDT”, “BTC”]
         
        Does it really matter?

## Notifiers

#### Activity

    channels:
       - text message 
       - discord 
       - telegram
       
    include: (per exchange)
        - coins added
        - coins removed
        - current balance
        - balance change (USD and BTC percent and quantity)
            - 1 day
            - 1 week
            - 1 month
            - 6 months
            - 1 year
            - All time
        - link to dashboard
    
#### Social

    channels:
        - Twitter
        - Instagram
        
    include:
        - select high proforming coin from last portfolio
        - image of trade
        
    referal links: ?
        - follow via shrimpy
        - sign up at exchange
      
#### System monitors
    
    channels:
       - text message
       
    include:
        - error
        - time